In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
csv_files_2017 = [('2017' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]
csv_files_2018 = [('2018' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]
csv_files_2019 = [('2019' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]

In [3]:
csv_files = csv_files_2017 + csv_files_2018 + csv_files_2019

In [4]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [6]:
def ignore_offpeak(df):
    df1 = df.copy()
    mask = df1['start_hour'].apply(lambda x: 5 <= x <= 20)
    return df1[mask]

In [9]:
for i, csv in enumerate(csv_files):
    df_temp = pd.read_csv('../Tripdata/'+csv)
    df_temp.columns = ['trip_duration','starttime','stoptime','start_station_ID','start_station_name',
                       'start_station_latitude','start_station_longitude','end_station_ID','end_station_name',
                       'end_station_latitude','end_station_longitude','bike_ID','user_type','birth_year','gender']
    df_temp = to_datetime(df_temp)
    df_temp = ignore_offpeak(df_temp)
    df_validate, df_train = train_test_split(df_temp, test_size=0.05, random_state=42 )
    df_notused, df_test = train_test_split(df_validate, test_size=0.05, random_state=42 )
    df_train.to_csv(months[i%12] + csv[:4] + 'train.csv')
    df_test.to_csv(months[i%12] + csv[:4] + 'test.csv')
    print('Finishing resetting ' + csv)

Finishing resetting 201701-citibike-tripdata.csv
Finishing resetting 201702-citibike-tripdata.csv
Finishing resetting 201703-citibike-tripdata.csv
Finishing resetting 201704-citibike-tripdata.csv
Finishing resetting 201705-citibike-tripdata.csv
Finishing resetting 201706-citibike-tripdata.csv
Finishing resetting 201707-citibike-tripdata.csv
Finishing resetting 201708-citibike-tripdata.csv
Finishing resetting 201709-citibike-tripdata.csv
Finishing resetting 201710-citibike-tripdata.csv
Finishing resetting 201711-citibike-tripdata.csv
Finishing resetting 201712-citibike-tripdata.csv
Finishing resetting 201801-citibike-tripdata.csv
Finishing resetting 201802-citibike-tripdata.csv
Finishing resetting 201803-citibike-tripdata.csv
Finishing resetting 201804-citibike-tripdata.csv
Finishing resetting 201805-citibike-tripdata.csv
Finishing resetting 201806-citibike-tripdata.csv
Finishing resetting 201807-citibike-tripdata.csv
Finishing resetting 201808-citibike-tripdata.csv
Finishing resetting 

In [10]:
train_list = []
test_list = []
for i, csv in enumerate(csv_files):
    train_list.append(months[i%12] + csv[:4] + 'train.csv')
    test_list.append(months[i%12] + csv[:4] + 'test.csv')

In [12]:
# Create a train dataset from 2017 to 2019 
train_df = pd.DataFrame()
for i in range(len(train_list)):
    temp_df = pd.read_csv(train_list[i], index_col = 0)
    train_df = pd.concat([train_df, temp_df], axis = 0)
train_df.to_csv('train.csv')

In [13]:
# Create a test dataset from 2017 to 2019 
test_df = pd.DataFrame()
for i in range(len(test_list)):
    temp_df = pd.read_csv(test_list[i], index_col = 0)
    test_df = pd.concat([test_df, temp_df], axis = 0)
test_df.to_csv('test.csv')

In [20]:
10e6

10000000.0

In [17]:
print("Train data set contains"+' ' +str(len(train_df))+' '+'Rows')
print("Test data set contains"+' ' +str(len(test_df))+' '+'Rows')

Train data set contains 2479389 Rows
Test data set contains 2355421 Rows


In [49]:
weather = pd.read_csv('weather.csv',index_col = 0)
weather = weather.fillna(0)
weather.reset_index(inplace=True)
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather

,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Fog,Heavy_Fog,Thunder,Hail,Glaze,Haze
0,2017-01-01,5.59,0.00,0.0,0.0,48,40,0.0,0.0,0.0,0.0,0.0,1.0
1,2017-01-02,9.17,0.21,0.0,0.0,41,37,1.0,0.0,0.0,1.0,0.0,0.0
2,2017-01-03,10.74,0.58,0.0,0.0,43,39,1.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-04,8.05,0.00,0.0,0.0,52,34,1.0,0.0,0.0,0.0,0.0,1.0
4,2017-01-05,7.83,0.00,0.0,0.0,34,27,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2019-12-27,4.47,0.00,0.0,0.0,54,46,0.0,0.0,0.0,0.0,0.0,0.0
1091,2019-12-28,4.03,0.00,0.0,0.0,51,43,1.0,0.0,0.0,0.0,0.0,1.0
1092,2019-12-29,4.92,0.25,0.0,0.0,44,39,1.0,0.0,0.0,0.0,0.0,0.0
1093,2019-12-30,12.75,0.74,0.0,0.0,41,37,1.0,0.0,0.0,0.0,0.0,0.0


In [50]:
train_df['start_date'] = pd.to_datetime(train_df['start_date'])

In [58]:
train_df_weather = train_df.merge(weather,how='left',left_on = 'start_date', right_on = 'DATE').drop(['DATE'],axis=1)

trip_duration                  0
starttime                      0
stoptime                       0
start_station_ID              90
start_station_name            90
start_station_latitude         0
start_station_longitude        0
end_station_ID                90
end_station_name              90
end_station_latitude           0
end_station_longitude          0
bike_ID                        0
user_type                    749
birth_year                 75189
gender                         0
start_date                     0
start_time                     0
start_hour                     0
start_year                     0
start_month                    0
start_dayofweek                0
stop_date                      0
stop_time                      0
stop_hour                      0
stop_dayofweek                 0
AWND                           0
PRCP                           0
SNOW                           0
SNWD                           0
TMAX                           0
TMIN      